In [1]:
import numpy as np
import pandas as pd
import os


In [2]:
# Get the current working directory
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

# List files in the current directory
files_in_directory = os.listdir(current_directory)
print(f"Files in directory: {files_in_directory}")

# Construct the full file path
# Replace 'actual_file_name.csv' with the actual name and extension of your file
file_path = os.path.join(current_directory, 'working file_mahendra.csv')
# or provide the absolute path like: file_path = '/path/to/your/file/actual_file_name.csv'

# Read the CSV file using the file path
df_print = pd.read_csv(file_path)

Current working directory: /content
Files in directory: ['.config', 'working file_mahendra.csv', 'sample_data']


<ipython-input-2-b30a53b4ddcb>:15: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_print = pd.read_csv(file_path)


In [3]:
df_print.shape

(1048575, 11)

In [4]:
df_clean = df_print[df_print['JobName'].notna() & (df_print['JobName'].str.strip() != '')]

In [5]:
df_clean.shape

(632958, 11)

In [27]:
df_sample = df_clean.sample(n=10000, random_state=42)
job_names_sample = df_sample['JobName'].astype(str).tolist()


In [25]:
#df_sample = df_sample.drop_duplicates(subset='JobName')

In [28]:
df_sample.shape

(10000, 11)

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(job_names_sample, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [9]:
import numpy as np

df_emb = pd.DataFrame(embeddings)
df_emb['job_name'] = job_names_sample  # keep job name attached


In [10]:
df_emb.shape

(10000, 385)

In [11]:
from sklearn.cluster import KMeans

X = df_emb.drop(columns='job_name').values
kmeans = KMeans(n_clusters=20, random_state=42)
labels = kmeans.fit_predict(X)

df_emb['cluster'] = labels


In [18]:
df_emb.columns

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          9,
       ...
              376,        377,        378,        379,        380,        381,
              382,        383, 'job_name',  'cluster'],
      dtype='object', length=386)

In [16]:
df_sample['cluster'] = df_emb['cluster'].values


In [17]:
df_sample.to_csv("clustered_job_names.csv", index=False)

In [29]:
# trying the clustering process with HDBSCAN

import hdbscan

# Safely drop non-embedding columns (job_name, KMeans cluster)
X_hdb = df_emb.drop(columns=['job_name', 'cluster']).values

# Run HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, metric='euclidean')
hdb_labels = clusterer.fit_predict(X_hdb)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [30]:
df_sample['hdbscan_cluster'] = hdb_labels


In [31]:
df_sample['hdbscan_cluster'].value_counts().sort_index()


,count
hdbscan_cluster,
-1,5503
0,100
1,127
2,218
3,179
4,113
5,1200
6,125
7,55


In [33]:
df_sample.to_csv("hdbscan_cluster1.csv", index=False)